Author: Meljohn Ugaddan</br>
Course : CSCI 273


In [169]:
!rm -r spark*
!wget https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!ls
!tar xvf ./spark-3.5.1-bin-hadoop3.tgz > /dev/null 2>/dev/null
!ls
!pip install -q findspark

--2024-04-19 16:54:26--  https://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400446614 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.1-bin-hadoop3.tgz’

spark-3.5.1-bin-had 100%[===================>] 381.90M  39.1MB/s    in 11s     

2024-04-19 16:54:37 (36.1 MB/s) - ‘spark-3.5.1-bin-hadoop3.tgz’ saved [400446614/400446614]

 archive.zip				   googleplaystore.gsheet   spark-3.5.1-bin-hadoop3.tgz
'Assignment 13 - Ugaddan, Meljohn.ipynb'   Input
 Getting_Started_with_Spark.ipynb	   Output
 archive.zip				   googleplaystore.gsheet   spark-3.5.1-bin-hadoop3
'Assignment 13 - Ugaddan, Meljohn.ipynb'   Input		    spark-3.5.1-bin-hadoop3.tgz
 Getting_Started_with_Spark.ipynb	   Output


In [170]:
# Set environment variables
import os
import logging
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3/"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-11-openjdk-amd64/jre/bin/java
!java -version

update-alternatives: error: alternative /usr/lib/jvm/java-11-openjdk-amd64/jre/bin/java for java not registered; not setting
openjdk version "11.0.22" 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)


In [171]:
DATA_PATH = '/content/drive/MyDrive/Big Data'
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization and mount your drive.
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Big Data
input_file1 = 'reviews.csv'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Big Data


In [172]:
DATA_PATH = '/content/drive/MyDrive/Big Data'

input_folder =  f"{DATA_PATH}/Input"
output_folder =  f"{DATA_PATH}/Output"

input_file1 = 'reviews.csv'

output_file1 = 'ranking_by_sum'
output_file2 = 'ranking_by_average'

In [173]:
ls

 archive.zip                               googleplaystore.gsheet   spark-3.5.1-bin-hadoop3/
'Assignment 13 - Ugaddan, Meljohn.ipynb'   Input/                   spark-3.5.1-bin-hadoop3.tgz
 Getting_Started_with_Spark.ipynb          Output/


In [174]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext
from operator import add
import pandas as pd

import csv
import re
import sys

In [175]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization and mount your drive.
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Big Data

conf = SparkConf().setMaster("local").setAppName("GooglePlayApps")
sc = SparkContext.getOrCreate(conf = conf)

def get_app_name(row):
    return row[-1], int(row[4])

reviews_rdd = sc.parallelize(pd.read_csv(f"{input_folder}/{input_file1}").values.tolist())
reviews_rdd.take(1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Big Data


[['gp:AOqpTOH0WP4IQKBZ2LrdNmFy_YmpPCVrV3diEU9KGm3fAX6VG0NAZCudCQpQRRI3GLL_tr8DQzUTP1hrOYG74A',
  'john alpha',
  'https://play-lh.googleusercontent.com/a-/AOh14GjpfgjOEbD3brypMeMT3KvhYlWG_nO2bMnMIfY9',
  'I have been begging for a refund from this app for over a month and nobody is replying me',
  1,
  0,
  nan,
  '2020-10-27 14:03:28',
  'Please note that from checking our records, your email has been answered, and there was no subscription registered under this address. Please check which email address you are using under you app settings- profile, and let us know so we can look further into this.',
  '2020-10-27 15:05:52',
  'newest',
  'com.anydo']]

In [176]:
reviews_score_rdd = reviews_rdd.map(lambda x: get_app_name(x))
reviews_score_rdd.take(5)

[('com.anydo', 1),
 ('com.anydo', 1),
 ('com.anydo', 1),
 ('com.anydo', 1),
 ('com.anydo', 1)]

### If the score is greater than 3 it'll be labeled as "Positive", if the rating is exactly 3 it'll be "Neutral", if its less than 3 it'll be "Negative"

In [177]:
review_dict_rdd = reviews_score_rdd.mapValues(lambda x:   {"Positive" : 1 if x > 3 else 0, "Neutral": 1 if x == 3 else 0 , "Negative": 1 if x < 3 else 0  } )
review_dict_rdd.take(5)

[('com.anydo', {'Positive': 0, 'Neutral': 0, 'Negative': 1}),
 ('com.anydo', {'Positive': 0, 'Neutral': 0, 'Negative': 1}),
 ('com.anydo', {'Positive': 0, 'Neutral': 0, 'Negative': 1}),
 ('com.anydo', {'Positive': 0, 'Neutral': 0, 'Negative': 1}),
 ('com.anydo', {'Positive': 0, 'Neutral': 0, 'Negative': 1})]

In [178]:
review_sum_score_rdd = reviews_score_rdd.reduceByKey(add)
review_sum_score_rdd.collect()

[('com.anydo', 2849),
 ('com.todoist', 2850),
 ('com.ticktick.task', 2850),
 ('com.habitrpg.android.habitica', 2850),
 ('cc.forestapp', 2850),
 ('com.oristats.habitbull', 2850),
 ('com.levor.liferpgtasks', 2290),
 ('com.habitnow', 1879),
 ('com.microsoft.todos', 2850),
 ('prox.lab.calclock', 2740),
 ('com.gmail.jmartindev.timetune', 2850),
 ('com.artfulagenda.app', 612),
 ('com.tasks.android', 2641),
 ('com.appgenix.bizcal', 2850),
 ('com.appxy.planner', 2850)]

In [179]:
def reduce_dicts(dict1, dict2):
    result = {}
    for key in dict1.keys():
        result[key] = dict1[key] + dict2.get(key, 0)
    for key in dict2.keys():
        if key not in dict1:
            result[key] = dict2[key]
    return result

sentiments_count = review_dict_rdd.reduceByKey(reduce_dicts)
sentiments_count.collect()

[('com.anydo', {'Positive': 400, 'Neutral': 150, 'Negative': 399}),
 ('com.todoist', {'Positive': 400, 'Neutral': 150, 'Negative': 400}),
 ('com.ticktick.task', {'Positive': 400, 'Neutral': 150, 'Negative': 400}),
 ('com.habitrpg.android.habitica',
  {'Positive': 400, 'Neutral': 150, 'Negative': 400}),
 ('cc.forestapp', {'Positive': 400, 'Neutral': 150, 'Negative': 400}),
 ('com.oristats.habitbull',
  {'Positive': 400, 'Neutral': 150, 'Negative': 400}),
 ('com.levor.liferpgtasks',
  {'Positive': 400, 'Neutral': 108, 'Negative': 117}),
 ('com.habitnow', {'Positive': 352, 'Neutral': 61, 'Negative': 64}),
 ('com.microsoft.todos', {'Positive': 400, 'Neutral': 150, 'Negative': 400}),
 ('prox.lab.calclock', {'Positive': 400, 'Neutral': 150, 'Negative': 345}),
 ('com.gmail.jmartindev.timetune',
  {'Positive': 400, 'Neutral': 150, 'Negative': 400}),
 ('com.artfulagenda.app', {'Positive': 102, 'Neutral': 22, 'Negative': 45}),
 ('com.tasks.android', {'Positive': 400, 'Neutral': 150, 'Negative': 

In [180]:
ranking_sum_of_scores = review_sum_score_rdd.sortBy(lambda x: x[1], ascending=False)
ranking_sum_of_scores.collect()

[('com.todoist', 2850),
 ('com.ticktick.task', 2850),
 ('com.habitrpg.android.habitica', 2850),
 ('cc.forestapp', 2850),
 ('com.oristats.habitbull', 2850),
 ('com.microsoft.todos', 2850),
 ('com.gmail.jmartindev.timetune', 2850),
 ('com.appgenix.bizcal', 2850),
 ('com.appxy.planner', 2850),
 ('com.anydo', 2849),
 ('prox.lab.calclock', 2740),
 ('com.tasks.android', 2641),
 ('com.levor.liferpgtasks', 2290),
 ('com.habitnow', 1879),
 ('com.artfulagenda.app', 612)]

In [181]:
ranking_by_sum_rdd = ranking_sum_of_scores.join(sentiments_count).sortBy(lambda x: x[1][0], ascending=False)
ranking_list = ranking_by_sum_rdd.take(10)

In [182]:
for rank_no,(key,val) in enumerate(ranking_list):
  ranking_score, sentiment_count = val
  app_name = key
  print(f" Rank {rank_no+1:2d}  App Name : {key.ljust(30)} \tRanking Score: {ranking_score}\tPositive Review Count: {sentiment_count['Positive']}\t Neutral Review Count: {sentiment_count['Neutral']}\t Negative Review Count: {sentiment_count['Negative']} ")

 Rank  1  App Name : com.ticktick.task              	Ranking Score: 2850	Positive Review Count: 400	 Neutral Review Count: 150	 Negative Review Count: 400 
 Rank  2  App Name : cc.forestapp                   	Ranking Score: 2850	Positive Review Count: 400	 Neutral Review Count: 150	 Negative Review Count: 400 
 Rank  3  App Name : com.oristats.habitbull         	Ranking Score: 2850	Positive Review Count: 400	 Neutral Review Count: 150	 Negative Review Count: 400 
 Rank  4  App Name : com.microsoft.todos            	Ranking Score: 2850	Positive Review Count: 400	 Neutral Review Count: 150	 Negative Review Count: 400 
 Rank  5  App Name : com.todoist                    	Ranking Score: 2850	Positive Review Count: 400	 Neutral Review Count: 150	 Negative Review Count: 400 
 Rank  6  App Name : com.habitrpg.android.habitica  	Ranking Score: 2850	Positive Review Count: 400	 Neutral Review Count: 150	 Negative Review Count: 400 
 Rank  7  App Name : com.gmail.jmartindev.timetune  	Ranking Sco

In [183]:
appname_count = reviews_score_rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
ranking_by_average_rdd = ranking_sum_of_scores.join(appname_count).mapValues(lambda x: x[0] / x[1]).join(sentiments_count).sortBy(lambda x: x[1][0], ascending=False)
ranking_by_average_list = ranking_by_average_rdd.take(10)


In [184]:
for rank_no,(key,val) in enumerate(ranking_by_average_list):
  ranking_score, sentiment_count = val
  app_name = key
  print(f" Rank {rank_no+1:2d}  App Name : {key.ljust(30)} \tRanking Score: {ranking_score:.4f}\tPositive Review Count: {sentiment_count['Positive']}\t Neutral Review Count: {sentiment_count['Neutral']}\t Negative Review Count: {sentiment_count['Negative']} ")

 Rank  1  App Name : com.habitnow                   	Ranking Score: 3.9392	Positive Review Count: 352	 Neutral Review Count: 61	 Negative Review Count: 64 
 Rank  2  App Name : com.levor.liferpgtasks         	Ranking Score: 3.6640	Positive Review Count: 400	 Neutral Review Count: 108	 Negative Review Count: 117 
 Rank  3  App Name : com.artfulagenda.app           	Ranking Score: 3.6213	Positive Review Count: 102	 Neutral Review Count: 22	 Negative Review Count: 45 
 Rank  4  App Name : com.tasks.android              	Ranking Score: 3.1858	Positive Review Count: 400	 Neutral Review Count: 150	 Negative Review Count: 279 
 Rank  5  App Name : prox.lab.calclock              	Ranking Score: 3.0615	Positive Review Count: 400	 Neutral Review Count: 150	 Negative Review Count: 345 
 Rank  6  App Name : com.anydo                      	Ranking Score: 3.0021	Positive Review Count: 400	 Neutral Review Count: 150	 Negative Review Count: 399 
 Rank  7  App Name : cc.forestapp                   	Ran

In [185]:
ranking_by_sum_rdd.saveAsTextFile(f"{output_folder}/{output_file1}")
ranking_by_average_rdd.saveAsTextFile(f"{output_folder}/{output_file2}")

In [186]:
sc.stop()